In [1]:
import numpy as np
import pinocchio as pin
from example_robot_data import load
from scipy.optimize import linprog
from scipy.spatial import ConvexHull

In [2]:
#loading go2 model
robot = load("go2")

In [3]:
#loading visualiser
from utils.meshcat_viewer_wrapper import MeshcatVisualizer # the meshcat visualiser
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
print(robot.model)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Nb joints = 14 (nq=19,nv=18)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 FL_hip_joint: parent=1
  Joint 3 FL_thigh_joint: parent=2
  Joint 4 FL_calf_joint: parent=3
  Joint 5 FR_hip_joint: parent=1
  Joint 6 FR_thigh_joint: parent=5
  Joint 7 FR_calf_joint: parent=6
  Joint 8 RL_hip_joint: parent=1
  Joint 9 RL_thigh_joint: parent=8
  Joint 10 RL_calf_joint: parent=9
  Joint 11 RR_hip_joint: parent=1
  Joint 12 RR_thigh_joint: parent=11
  Joint 13 RR_calf_joint: parent=12



In [4]:
#sampling a random configuration
q = pin.randomConfiguration(robot.model)
viz.display(q)

In [5]:
print(robot.model)

Nb joints = 14 (nq=19,nv=18)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 FL_hip_joint: parent=1
  Joint 3 FL_thigh_joint: parent=2
  Joint 4 FL_calf_joint: parent=3
  Joint 5 FR_hip_joint: parent=1
  Joint 6 FR_thigh_joint: parent=5
  Joint 7 FR_calf_joint: parent=6
  Joint 8 RL_hip_joint: parent=1
  Joint 9 RL_thigh_joint: parent=8
  Joint 10 RL_calf_joint: parent=9
  Joint 11 RR_hip_joint: parent=1
  Joint 12 RR_thigh_joint: parent=11
  Joint 13 RR_calf_joint: parent=12



In [6]:
#defining effector frames

pin.forwardKinematics(robot.model, robot.data, robot.q0)
pin.updateFramePlacements(robot.model, robot.data)

effectors = ["FL_foot_joint", "RL_foot_joint", "RR_foot_joint", "FR_foot_joint"]
frame_ids = [robot.model.getFrameId(eff) for eff in effectors]
placements = [robot.data.oMf[frame_id] for frame_id in frame_ids]

#display effectors positions just to make sure that there is no offset
effectorspheres_id = ['world/ball'+eff for eff in effectors]
[viz.addSphere(sph_id,.023,[1,0,0,1]) for sph_id in effectorspheres_id]

for frame_id, sph_id in zip(frame_ids, effectorspheres_id):
    pos = robot.data.oMf[frame_id].translation
    viz.applyConfiguration(sph_id,[pos[0],pos[1],pos[2],1,0,0,0])


In [7]:
NUM_SAMPLES = 18000
IT_DISPLAY_PROGRESS = NUM_SAMPLES / 10
MIN_DIST_BETWEEN_FEET_Y = 0.10
MIN_DIST_BETWEEN_FEET_X = 0.10
MAX_DIST_BETWEEN_FEET_X = 0.35
MAX_DIST_BETWEEN_FEET_Z = 0.35
MIN_HEIGHT_COM = 0.3
# margin used to constrain the com y position : if it's on the left of the left foot
# or on the right of the right foot
# for more than this margin, we reject this sample:
MARGIN_FEET_SIDE = 0.02
ZERO_CONF = robot.q0[:7]

In [8]:
[pl.translation for pl in placements]

[array([0.17259716, 0.16349486, 0.0226347 ]),
 array([-0.21420284,  0.16349486,  0.0226347 ]),
 array([-0.21420284, -0.16349486,  0.0226347 ]),
 array([ 0.17259716, -0.16349486,  0.0226347 ])]

In [24]:
#define a function that generates a random configuration that is acceptable, ie
# collision free, with the trunk above the feet, no cross over, and somehow in quasi static equilibrium

#applies a configuration and updates geometry and data
def apply(q):    
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, robot.collision_data)

# all functions below assume apply has been called
def collision():
    return pin.computeCollisions(robot.collision_model,robot.collision_data,True)
    
def feet_below_base_ok(positions):
    """
    returns true if the configuration is in quasi-static equilibrium assuming quasi-flat surfaces
    """
    return all( [pos[2] < ZERO_CONF[2]-0.1 for pos in positions])
    
def no_crossing(positions):
    """
    returns true if legs do not cross
    """
    #order is effectors = ["FL_foot_joint", "RL_foot_joint", "RR_foot_joint", "FR_foot_joint"]
    # first, front legs before back legs
    if positions[0][0] <  positions[1][0] or positions[2][0] >  positions[3][0]:
        return False
    # then left leg more to the right than right leg
    if positions[0][1] <  positions[3][1] or positions[1][1] <  positions[2][1]:
        return False
    return True
    

    
def stay_on_side(positions):
    """
    returns true if legs stay on their "side"
    """
    #order is effectors = ["FL_foot_joint", "RL_foot_joint", "RR_foot_joint", "FR_foot_joint"]
    # first, front legs before back legs
    if positions[0][1] <  MARGIN_FEET_SIDE or positions[1][1]  <  MARGIN_FEET_SIDE:
        return False
    # then left leg more to the right than right leg
    if positions[2][1] >  -MARGIN_FEET_SIDE or positions[3][1]  >  -MARGIN_FEET_SIDE:
        return False
    return True
    
def static_eq(positions, com):    
    """
    returns true if the configuration is in quasi-static equilibrium assuming quasi-flat surfaces
    """
    return True;
    sizeX = len(positions)
    E = np.zeros((3, sizeX))
    for i, pos in enumerate(positions):
        E[:2, i] = pos[:2]
    e = np.array([com[0], com[1], 1.0])
    E[2, :] = np.ones(sizeX)
    res = linprog(
        np.ones(sizeX),
        A_ub=None,
        b_ub=None,
        A_eq=E,
        b_eq=e,
        bounds=[(0.0, 1.0) for _ in range(sizeX)],
        method="interior-point",
        callback=None,
        options={"presolve": True},
    )
    return res["success"]



def point_not_inside_hull(positions):
    """
    returns False of one of the point is inside the convex hulls of the others.
    We do not want that, or do we?
    """
    for i, pos in enumerate(positions):
        others = positions[:i] + positions[i + 1 :]
        if staticEq(others, pos):
            return False
    return True

def gen_quasi_static():
    while True:
        q = pin.randomConfiguration(robot.model)
        q[:7]= ZERO_CONF[:]
        apply(q)    
        for frame_id, sph_id in zip(frame_ids, effectorspheres_id):
            pos = robot.data.oMf[frame_id].translation
        positions = [robot.data.oMf[frame_id].translation for frame_id in frame_ids]
        com = pin.centerOfMass(robot.model, robot.data, q)
        if not collision() and stay_on_side(positions) and no_crossing(positions) and feet_below_base_ok(positions) and static_eq(positions, com):
            return q
    

q = gen_quasi_static()
viz.display(q)
for frame_id, sph_id in zip(frame_ids, effectorspheres_id):
    pos = robot.data.oMf[frame_id].translation
    viz.applyConfiguration(sph_id,[pos[0],pos[1],pos[2],1,0,0,0])

In [21]:
positions

NameError: name 'positions' is not defined

In [16]:
#storing com points and other effectors points
compoints = [[] for _ in effectors]
points = [{} for _ in effectors]
for i, eff in enumerate(effectors):
    for j, otherEff in enumerate(effectors):
        if i != j:
            points[i][otherEff] = []
            

In [ ]:
def printFootPositionRelativeToOther(q_s, pos_s, com_s):
    nbConfigs = len(q_s)
    for i, (q, pos, com) in enumerate(zip(q_s, pos_s, com_s)):
        if i > 0 and not i % IT_DISPLAY_PROGRESS:
            print(int((i * 100) / nbConfigs), " % done")
            success += 1
            addCom = True
            for j, effectorName in enumerate(effectors):
                for otheridx, (oeffectorName, limbId) in enumerate(
                    zip(effectors, limbIds)
                ):
                    if otheridx != j:
                        fullBody.setCurrentConfig(q)
                        pos_other = fullBody.getJointPosition(oeffectorName)
                        pos = fullBody.getJointPosition(effectorName)
                        p = array(pos_other[:3]) - array(pos[:3]).tolist()
                        if MAX_DIST_BETWEEN_FEET_Z > abs(p[2]):
                            if MIN_DIST_BETWEEN_FEET_Y <= abs(p[1]):
                                if MIN_DIST_BETWEEN_FEET_X <= abs(p[0]):
                                    # this is not what we want to do in theory
                                    # but it works well in fact
                                    points[j][oeffectorName].append(p[:3])
                                else:
                                    addCom = False
                            else:
                                addCom = False
                        else:
                            print(
                                "rejecting ",
                                effectorName,
                                " ",
                                oeffectorName,
                                p,
                                abs(p[2]),
                            )
                            # ~ print ('pos_other', pos_other)
                            # ~ print ('old_pos', old_pos)
                            addCom = False
                            v(q)
                        # ~ if (j == 0 and p[1] > MIN_DIST_BETWEEN_FEET_Y
                        #       and abs(p[0]) < MAX_DIST_BETWEEN_FEET_X):
                        # ~ points[j].append(p[:3])
                        # ~ elif (j == 1 and p[1] < -MIN_DIST_BETWEEN_FEET_Y
                        #         and abs(p[0]) < MAX_DIST_BETWEEN_FEET_X):
                        # ~ points[j].append(p[:3])
                        # ~ else:
                        # ~ addCom =
            # now compute coms

            fullBody.setCurrentConfig(q)
            com = array(fullBody.getCenterOfMass())
            print("com ", com)
            # ~ for x in range(0, 3):
            # ~ q[x] = -com[x]
            for j, effectorName in enumerate(effectors):
                pos = fullBody.getJointPosition(effectorName)
                rp = array(com) - array(pos[:3]).tolist()
                # ~ qEffector = fullBody.getJointPosition(effectorName)
                # ~ q0 = Quaternion(qEffector[6], qEffector[3], qEffector[4],
                #                   qEffector[5])
                # ~ rot = q0.matrix()  # compute rotation matrix world -> local
                # ~ p = qEffector[0:3]  # (0,0,0) coordinate expressed in effector fram
                # ~ rm = np.zeros((4, 4))
                # ~ for k in range(0, 3):
                # ~ for l in range(0, 3):
                # ~ rm[k, l] = rot[k, l]
                # ~ for m in range(0, 3):
                # ~ rm[m, 3] = qEffector[m]
                # ~ rm[3, 3] = 1
                # ~ invrm = np.linalg.inv(rm)
                # ~ p = invrm.dot([0, 0, 0, 1])
                # ~ # add offset
                # ~ rp = array(p[:3] - offsets[j]).tolist()

                if rp[2] < MIN_HEIGHT_COM:
                    addCom = False
                    print("reject min heught")
                if addCom:
                    compoints[j].append(rp)
                    # ~ if j == 1:
                    # ~ if rp[1] < MARGIN_FEET_SIDE:
                    # ~ compoints[j].append(rp)
                    # ~ else:
                    # ~ if rp[1] > -MARGIN_FEET_SIDE:
                    # ~ compoints[j].append(rp)


In [140]:
NUM_SAMPLES = 18000
IT_DISPLAY_PROGRESS = NUM_SAMPLES / 10
MIN_DIST_BETWEEN_FEET_Y = 0.10
MIN_DIST_BETWEEN_FEET_X = 0.10
MAX_DIST_BETWEEN_FEET_X = 0.35
MAX_DIST_BETWEEN_FEET_Z = 0.35
MIN_HEIGHT_COM = 0.3
# margin used to constrain the com y position : if it's on the left of the left foot
# or on the right of the right foot
# for more than this margin, we reject this sample:
MARGIN_FEET_SIDE = 0.05
ZERO_CONF = robot.q0[:7]

In [58]:
for joint_id, sph_id in zip(joints_ids, effectorspheres_id):
    pos = robot.data.oMf[joint_id].translation
    print("pos", pos)
    viz.applyConfiguration(sph_id,[pos[0],pos[1],pos[2],1,0,0,0])

pos [-0.56475879  0.40738133  0.18074512]
pos [0.0830834  0.42975445 0.36444446]
pos [-0.01686018  0.91640843  0.3697123 ]
pos [-0.15634456  0.32184715 -0.42638796]


In [83]:
robot.q0[:7]

array([0.   , 0.   , 0.335, 0.   , 0.   , 0.   , 1.   ])